In [ ]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'])

In [ ]:
train_dataset = SoilDataset(train_df, '/kaggle/input/soil-classification/soil_classification-2025/train', transform)
val_dataset   = SoilDataset(val_df, '/kaggle/input/soil-classification/soil_classification-2025/train', transform)

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, 4)  # 4 soil types
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
def train(model, loader):
    model.train()
    total_loss = 0
    for imgs, labels in tqdm(loader):
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

In [ ]:
def evaluate(model, loader):
    model.eval()
    preds, true = [], []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)
            preds.extend(predicted.cpu().numpy())
            true.extend(labels.numpy())
    return f1_score(true, preds, average=None), classification_report(true, preds, target_names=class_names)

In [ ]:
# Epochs
for epoch in range(10):
    train_loss = train(model, train_loader)
    f1, report = evaluate(model, val_loader)
    print(f"Epoch {epoch+1} - Loss: {train_loss:.4f}")
    print("F1 per class:", f1)
    print("Min F1:", f1.min())
    print(report)